In [2]:
import re
with open("the-verdict.txt", "r") as file:
    raw_text = file.read()

preprocessed = re.split(r'([,.;:?_"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [34]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab = {token: integer for integer, token in enumerate(all_words)}
print(vocab_size)
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 10:
        break

1148
('"', 0)
("'", 1)
('(', 2)
(')', 3)
(',', 4)
('--', 5)
('.', 6)
(':', 7)
(';', 8)
('?', 9)
('A', 10)


In [12]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str2id = vocab
        self.id2str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'[, .?_!"\'()|\s]', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = []
        for s in preprocessed:
            if s in self.str2id:
                ids.append(self.str2id[s])
            else:
                print(f"警告: 词汇 '{s}' 在词汇表中没有找到")
                ids.append(-1)  # 可以给未知词汇一个默认值，或者做其他处理
        return ids

    def decode(self, ids):
        text = " ".join(self.id2str.get(i, '[UNK]') for i in ids)  # 对于未知的 ID，使用 [UNK]
        text = re.sub(r'\s+([, .?_!"\'()|\s])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV1(vocab)
text = raw_text[:99]
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

警告: 词汇 'genius--though' 在词汇表中没有找到
警告: 词汇 'enough--so' 在词汇表中没有找到
[53, 44, 154, 1016, 57, 37, 829, 120, 261, -1, 120, 505, 440, -1, 592, 1091, 717]
I HAD always thought Jack Gisburn rather a cheap [UNK] a good fellow [UNK] it was no


In [24]:
text = "genius--though"
print(tokenizer.encode(text))

警告: 词汇 'genius--though' 在词汇表中没有找到
[-1]


In [28]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [35]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [36]:
enc_sample = enc_text[50:]
context_size = 4
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [37]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(text)
        for i in range(0, len(token_ids) - max_length, stride):
            input = token_ids[i:i+max_length]
            target = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input))
            self.target_ids.append(torch.tensor(target))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
dataset = GPTDatasetV1(raw_text, tokenizer, max_length=4, stride=1)

dataloader = DataLoader(dataset, batch_size=3, shuffle=False)

for i, (input, target) in enumerate(dataloader):
    if i > 2:
        break
    print(f'Batch {i+1} Input: {input}')
    print(f'Batch {i+1} Target: {target}')

Batch 1 Input: tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619]])
Batch 1 Target: tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402]])
Batch 2 Input: tensor([[ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899]])
Batch 2 Target: tensor([[ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138]])
Batch 3 Input: tensor([[  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026]])
Batch 3 Target: tensor([[  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026],
        [ 2138,   257,  7026, 15632]])
